# Test 03 — Line sensors (sequential prove)\n\nGoal: prove each of the 4 IR sensors can read True.\nIt will prompt you: "Move sensor N over black tape".\nIt retries I2C reads and won't drive the robot.\n

In [ ]:
from pathlib import Path
import sys

def add_repo_to_path():
    here = Path.cwd().resolve()
    for p in [here] + list(here.parents):
        if (p / 'lessons').is_dir() and (p / 'common').is_dir():
            repo_root = p
            common_lib = p / 'common' / 'lib'
            if str(common_lib) not in sys.path:
                sys.path.insert(0, str(common_lib))
            print('Repo root:', repo_root)
            print('Added common/lib:', common_lib)
            return repo_root
    raise FileNotFoundError('Could not find repo root (needs lessons/ and common/)')

add_repo_to_path()

In [ ]:
import time
from fast_sdk.infra_red import InfraredSensors

ir = InfraredSensors()
ir.address = 0x77  # adjust if your scan differs
print('IR addr:', hex(ir.address))

def safe_read(retries=6, pause_s=0.03):
    last = None
    for _ in range(retries):
        try:
            vals = ir.read_sensor_data()  # List[bool]
            if vals and len(vals) == 4:
                return vals
            last = vals
        except OSError as e:
            last = e
            time.sleep(pause_s)
    raise RuntimeError(f'I2C read failed after {retries} tries: {last}')

def bar(vals):
    return ''.join('■' if v else '·' for v in vals)

print('Reading once:', bar(safe_read()), safe_read())


In [ ]:
targets = [0, 1, 2, 3]

for idx in targets:
    print('\n---')
    print(f'Target sensor {idx+1}/4. Move THAT sensor over black tape until it reads True.')
    start = time.time()
    ok = False
    while time.time() - start < 30:
        vals = safe_read()
        print(bar(vals), vals)
        if vals[idx] is True:
            ok = True
            break
        time.sleep(0.12)
    if not ok:
        raise RuntimeError(f'Sensor {idx+1} did not go True within 30s. Check wiring/height/threshold.')
    print(f'✅ Sensor {idx+1} proven')

print('\nAll sensors proven ✅')
